In [6]:
import pandas as pd
def data_from_histogram(num_data, lim_inf,lim_sup,lista):
  for i in range(num_data):
    lista.append(np.random.uniform(lim_inf,lim_sup))
  return lista

lista = []
lista = data_from_histogram(2,90,95,lista)
lista = data_from_histogram(8,95,100,lista)
lista = data_from_histogram(5,100,105,lista)
lista = data_from_histogram(4,105,110,lista)
lista = data_from_histogram(6,110,115,lista)
lista = data_from_histogram(5,115,120,lista)
data2 = pd.Series(lista)

NameError: name 'pd' is not defined

In [ ]:
def main2():
    #Vamos a generar los datos como la tabla dada.
    data = pd.DataFrame(data2)
    # data.hist(bins = range(90,121,5))
    table, bins = create_freq_dist_table2(data)
    stats = data_from_table(table)
    d = pd.DataFrame({'': ['Media','Standard Deviation','Minimum','25%','50%','75%','Maximum','Skew','Kurtosis'],"data":stats})
    print(d)
main2()